# EXAMEN FINAL DE PROGRAMACIÓN LINEAL Y FLUJOS EN REDES

In [1]:
import sys
sys.path.insert(0, "..")

from algorithms.network_flows import *
from algorithms.util2 import print_model
from algorithms.simplex_algorithms import *
import numpy as np
import sympy as sp

In [2]:
def build_matrix(number_node: int, edges: list[Edge]):
    # Output: matrix mx(n+1) A, with component i=1, j=-1, and last component 1
    number_edge = len(edges)
    A = sp.zeros(number_node, number_edge)
    for k in range(number_edge - 1):
        A[edges[k].i - 1, k] = 1
        A[edges[k].j - 1, k] = -1
    A[edges[-1].i - 1, -1] = 1
    return A


#### 1. Resolver el siguiente problema de flujo en red de costo mínimo con valores bi en los nodos y valores cij en los arcos. (Utilizando la teoría de grafos y árboles). (8 ptos)


In [3]:
number_nodes = 4
edges: list[Edge] = [Edge(1, 2, 3),
                     Edge(1, 3, 3),
                     Edge(2, 4, 1),
                     Edge(2, 3, 4),
                     Edge(3, 2, -2),
                     Edge(3, 4, -5),
                     Edge(4, np.Inf, 0)]

A = build_matrix(number_nodes, edges)
b = np.array([1, 0, 0, -1])  # sum(b) = 0
c = np.array(list(map(lambda e: e.cost, edges)))
var = set(range(len(edges)))


In [4]:
print_model(A, b, c, "x", get_edges(edges, list(var)), ["="] * number_nodes, "min")


In [5]:
I = sp.Matrix([[1, 0, 0, 0],
               [0, 1, 0, 0],
               [0, 0, 1, 0],
               [0, 0, 0, 1]])
print_model(I, b, [-1, -1, -1, -1], "u", [1, 2, 3, 4], ["="] * number_nodes, "max")


In [6]:
A = np.array(A).astype(float)
basic_var = [1, 2, 4, 6]  # solucion [1, 3, 5, 6]
basic_edges = get_edges(edges, basic_var)
print(f"variables básicas\nBI = {basic_var}")
print_vector("map", "x", [r"x_{%s}" % i for i in basic_var], basic_edges)

B = A[:, basic_var]
print(f"Matriz B\n{B}")
x_sol = np.linalg.inv(B) @ b
print_vector("solución básica (flujos)", "x", x_sol, basic_edges)

# solución dual
w = np.linalg.inv(B).T @ c[basic_var]
# en el dibujo de flujo en redes, situo la wi en la cola del enlace aij
print_vector("variable dual", "w", w, list(range(1, len(w) + 1)))

non_basic_var = list(var - set(basic_var))
non_basic_edges = get_edges(edges, non_basic_var)
# aún no es negativo, lo cambiamos
z = w @ A[:, non_basic_var] - c[non_basic_var]
print_vector("costos no básicos", "w", z, non_basic_edges)


variables básicas
BI = [1, 2, 4, 6]


Matriz B
[[ 1.  0.  0.  0.]
 [ 0.  1. -1.  0.]
 [-1.  0.  1.  0.]
 [ 0. -1.  0.  1.]]


In [7]:
A = np.array(A).astype(float)
basic_var = [1, 3, 5, 6]  # [2, 4, 6, 7]
basic_edges = get_edges(edges, basic_var)
print(f"variables básicas\nBI = {basic_var}")
print_vector("map", "x", [r"x_{%s}" % i for i in basic_var], basic_edges)

B = A[:, basic_var]
print(f"Matriz B\n{B}")
x_sol = np.linalg.inv(B) @ b
print_vector("solución básica (flujos)", "x", x_sol, basic_edges)

# solución dual
w = np.linalg.inv(B).T @ c[basic_var]
# en el dibujo de flujo en redes, situo la wi en la cola del enlace aij
print_vector("variable dual", "w", w, list(range(1, len(w) + 1)))

non_basic_var = list(var - set(basic_var))
non_basic_edges = get_edges(edges, non_basic_var)
# aún no es negativo, lo cambiamos
z = w @ A[:, non_basic_var] - c[non_basic_var]
print_vector("costos no básicos", "w", z, non_basic_edges)


variables básicas
BI = [1, 3, 5, 6]


Matriz B
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [-1. -1.  1.  0.]
 [ 0.  0. -1.  1.]]


In [8]:
x_full = np.zeros_like(c)
x_full[basic_var] = x_sol
z = np.dot(c, x_full)
print("z* =", z)


z* = -2


#### 2. Modele el problema 1 como un problema de programación lineal:


a) Ponerlo en la forma estandar y resolverlo utilizando el método simplex (Use Big M o dos fases para hallar una primera solución básica factible).(4 ptos)

In [9]:
A = np.array([[1., 1.0, 0., 0., 0.0, 0., 0., 1, 0, 0, 0],
              [-1., 0., 1., 1., -1., 0., 0., 0, 1, 0, 0],
              [0., -1., 0., -1., 1., 1., 0., 0, 0, 1, 0],
              [0., 0., -1., 0., 0., -1., 1., 0, 0, 0, 1]])

c_max = -np.array([3, 3, 1, 4, -2, -5, 0])

c_artificial = [0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1.0]

basic_index = [7, 8, 9, 10]
artificial_index = [7, 8, 9, 10]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c_max, basic_index, artificial_index)


x_B = [8, 9, 10, 11]
[[ 0.  0.  0.  0.  0.  0.  0. -1. -1. -1. -1.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
Cost corrected
x_B = [8, 9, 10, 11]
[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
Start phase One
x_B = [8, 9, 10, 7]
[[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0. -1.  1.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
x_B = [8, 3, 10, 7]
[[ 1.  0.  0. -1.  1.  1.  0.  0. -1.  0. -1.  1.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  1.]
 [-1.  0.  1.  1.

b) Halle el modelo dual del problema de programación lineal y resuelvalo. (4 ptos)

In [10]:
number_nodes = 4
edges: list[Edge] = [Edge(1, 2, 3),
                     Edge(1, 3, 3),
                     Edge(2, 4, 1),
                     Edge(2, 3, 4),
                     Edge(3, 2, -2),
                     Edge(3, 4, -5),
                     Edge(4, np.Inf, 0)]

A = build_matrix(number_nodes, edges)
b = np.array([1, 0, 0, -1])  # sum(b) = 0
c = np.array(list(map(lambda e: e.cost, edges)))
var = set(range(len(edges)))


In [11]:
from itertools import combinations
from IPython.display import display_latex, Latex


def pretty_print(string: str):
    return display_latex(Latex(string))


In [12]:
import math


In [13]:
def factible_vertex(A: sp.Matrix, b: sp.Matrix, c: sp.Matrix):
    m, n = A.shape
    comb = tuple(combinations(range(m), n))
    print("Number of combinations:", len(comb))
    # for each combiantion for indexes
    for indexes in comb:
        try:
            A_inv = sp.Matrix.inv(A.row(indexes))
            x = A_inv @ b.row(indexes)
            show = True
            # for each constraint
            for j in range(m):
                if not A.row(j).dot(x) <= b[j]:
                    show = False
                    break
            if show:
                pretty_print(f"$H={sp.latex(A.row(indexes))}, H^{{-1}}={sp.latex(A_inv)}\quad x{indexes}={sp.latex(x)}\quad c={c.dot(x).evalf()}$")
        except Exception as ex:
            print(ex)
            continue


In [14]:
A_T = A.T
b = sp.Matrix([1, 0, 0, -1])
c = sp.Matrix([3, 3, 1, 4, -2, 5, 0])
factible_vertex(A_T, c, b)


Number of combinations: 35
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.


Matrix det == 0; not invertible.


Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.
Matrix det == 0; not invertible.


In [15]:
A_T = A.T
print_model(A_T, c, b, "y", list(range(1, A_T.shape[1] + 1)), ["\le"] * A_T.shape[0], "max")
# obs yi es free!!!


c) ¿Qué pasa con la función objetivo si cada bi es perturbado por un $δ$?. Halle los intervalos de tolerancia para cada caso.(4 ptos)


In [16]:
number_nodes = 4
edges: list[Edge] = [Edge(1, 2, 3),
                     Edge(1, 3, 3),
                     Edge(2, 4, 1),
                     Edge(2, 3, 4),
                     Edge(3, 2, -2),
                     Edge(3, 4, -5),
                     Edge(4, np.Inf, 0)]

A = build_matrix(number_nodes, edges)
b = np.array([1, 0, 0, -1])  # sum(b) = 0
c = np.array(list(map(lambda e: e.cost, edges)))
var = set(range(len(edges)))


In [17]:
b


array([ 1,  0,  0, -1])

In [18]:
def pivoting_symbolic(tableau: np.ndarray, row: int, col: int) -> np.ndarray:
    # escale pivot row min to 1.0
    tableau[row, :] = tableau[row, :] / tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(tableau.shape[0]):
        if k != row:
            tableau[k, :] = tableau[k, :] - tableau[k, col] * tableau[row, :]


def correct_symbolic_cost(tableau: np.ndarray, basic_var: list[int]) -> None:
    # correct basic variable cost with value distict of zero
    for index, col in enumerate(basic_var):
        row = index + 1
        tableau[0, :] = tableau[0, :] - tableau[0, col] * tableau[row, :]
    print(f"Cost corrected")


In [19]:
number_nodes = 4
edges: list[Edge] = [Edge(1, 2, 3),
                     Edge(1, 3, 3),
                     Edge(2, 4, 1),
                     Edge(2, 3, 4),
                     Edge(3, 2, -2),
                     Edge(3, 4, -5),
                     Edge(4, np.Inf, 0)]

A = build_matrix(number_nodes, edges)
b = np.array([1, 0, 0, -1])  # sum(b) = 0
c = np.array(list(map(lambda e: e.cost, edges)))
var = set(range(len(edges)))


In [20]:
A = np.array([[1., 1.0, 0., 0., 0.0, 0., 0., 1, 0, 0, 0],
              [-1., 0., 1., 1., -1., 0., 0., 0, 1, 0, 0],
              [0., -1., 0., -1., 1., 1., 0., 0, 0, 1, 0],
              [0., 0., -1., 0., 0., -1., 1., 0, 0, 0, 1]])

c_max = -np.array([3, 3, 1, 4, -2, -5, 0])
b = np.array([ 1 + -1,  0,  0, -1])
c_artificial = [0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1.0]

basic_index = [7, 8, 9, 10]
artificial_index = [7, 8, 9, 10]

tableau = build_tableau(A, b, c_artificial)
tableau = two_phases(tableau, c_max, basic_index, artificial_index)


x_B = [8, 9, 10, 11]
[[ 0.  0.  0.  0.  0.  0.  0. -1. -1. -1. -1.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
Cost corrected
x_B = [8, 9, 10, 11]
[[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0. -1.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
Start phase One
x_B = [8, 9, 10, 7]
[[ 0.  0.  1.  0.  0.  1.  0.  0.  0.  0. -1.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  1. -1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0. -1.  0. -1.  1.  1.  0.  0.  0.  1.  0.  0.]
 [ 0.  0. -1.  0.  0. -1.  1.  0.  0.  0.  1. -1.]]
x_B = [8, 3, 10, 7]
[[ 1.  0.  0. -1.  1.  1.  0.  0. -1.  0. -1.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [-1.  0.  1.  1.